In [ ]:
# load all units for movie/shuffled stationary and for spontaneous stationary
# take activity averages across time and compare unit activity
# filter units based on activity
# create report similar to sfn_panel3

# Unit Activity Report

In [3]:
# imports
import numpy as np
import pandas as pd
import neo

print("Packages Imported!")

Packages Imported!


### Load units

In [6]:
PROJECT_PATH = "G:/Shared drives/visual_encoding"
dir_input = f"{PROJECT_PATH}/data/blocks/sessions" 

In [8]:
block = neo.io.NeoMatlabIO(filename=f"{dir_input}/block_766640955.mat").read_block()

In [13]:
list(block.segments[0].spiketrains)

[SpikeTrain
 name: '950929283'
 annotations: {'unit_id': 950929283,
   'probe_id': 773592320,
   'channel_id': 850025328,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929267'
 annotations: {'unit_id': 950929267,
   'probe_id': 773592320,
   'channel_id': 850025328,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929299'
 annotations: {'unit_id': 950929299,
   'probe_id': 773592320,
   'channel_id': 850025330,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950931534'
 annotations: {'unit_id': 950931534,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929433'
 annotations: {'unit_id': 950929433,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929417'
 annotations: {'unit_id': 950929417,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950931556'
 annotations: {'unit_id': 950931556,
   'probe_id':

In [38]:
dir_spontaneous_stationary = f"{PROJECT_PATH}/data/blocks/segmented/spontaneous_stationary"
dir_movie = f"{PROJECT_PATH}/data/blocks/segmented/natural_movie_one_more_repeats"
dir_shuffled = f"{PROJECT_PATH}/data/blocks/segmented/natural_movie_one_shuffled"

spont_block = neo.io.NeoMatlabIO(filename=f"{dir_spontaneous_stationary}/block_766640955.mat").read_block()
movie_block = neo.io.NeoMatlabIO(filename=f"{dir_movie}/block_766640955.mat").read_block()
shuffled_block = neo.io.NeoMatlabIO(filename=f"{dir_shuffled}/block_766640955.mat").read_block()


In [ ]:
# for each session
    # load spont block, get first segment
    # for each unit
        # for each segment in spont
            # average firing rate and store
            
        # for each segment in movie
            # average firing rate and store
            
        # for each segment in shuffled
            # average firing rate and store
            
        # average movie and shuffled rates and store
            
        # if spont >= stim
            # provide print statement
            # delete unit from shuffled and stim blocks

# save out filtered block

In [ ]:
dir_spontaneous_stationary = f"{PROJECT_PATH}/data/blocks/segmented/spontaneous_stationary"
dir_movie = f"{PROJECT_PATH}/data/blocks/segmented/natural_movie_one_more_repeats"
dir_shuffled = f"{PROJECT_PATH}/data/blocks/segmented/natural_movie_one_shuffled"

for file in os.listdir(dir_movie):
    
    spont_block = neo.io.NeoMatlabIO(filename=f"{dir_spontaneous_stationary}/{file}").read_block()
    movie_block = neo.io.NeoMatlabIO(filename=f"{dir_movie}/{file}").read_block()
    shuff_block = neo.io.NeoMatlabIO(filename=f"{dir_shuffled}/{file}").read_block()
    session_id = file.split('_')[1]
    
    for i_train, spiketrain in enumerate(spont_block.segments[0].spiketrains):
        
        unit_id = spiketrain.annotations['unit_id']
        
        spont_avg = 0
        stim_avg = 0
        
        for segment in spont_block.segments:
            
            spikes = segment.spiketrains[i_train]
            assert spikes['unit_id'] == unit_id
            
            mfr = (len(spikes)/spikes.duration).magnitude
            spont_avg = (spont_avg + mfr)/2
            
        for segment in movie_block.segments:
            
            spikes = segment.spiketrains[i_train]
            assert spikes['unit_id'] == unit_id
            
            mfr = (len(spikes)/spikes.duration).magnitude
            stim_avg = (stim_avg + mfr)/2
            
        for segment in shuff_block.segments:
            
            spikes = segment.spiketrains[i_train]
            assert spikes['unit_id'] == unit_id
            
            mfr = (len(spikes)/spikes.duration).magnitude
            stim_avg = (stim_avg + mfr)/2
        
        if spont_avg > stim_avg:
            
            for segment in movie_block.segments:
                
                segment.spiketrains = segment.spiketrains[:i_train] + 
                segment.spiketrains[i_train+1:]
            

            for segment in shuff_block.segments:

                segment.spiketrains = segment.spiketrains[:i_train] + 
                segment.spiketrains[i_train+1:]
                
# save out block

In [69]:
block.segments[0].spiketrains[5]

SpikeTrain
name: '950929417'
annotations: {'unit_id': 950929417,
  'probe_id': 773592320,
  'channel_id': 850025334,
  'brain_structure': 'VISp'}

In [70]:
list(block.segments[0].spiketrains)

[SpikeTrain
 name: '950929283'
 annotations: {'unit_id': 950929283,
   'probe_id': 773592320,
   'channel_id': 850025328,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929267'
 annotations: {'unit_id': 950929267,
   'probe_id': 773592320,
   'channel_id': 850025328,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929299'
 annotations: {'unit_id': 950929299,
   'probe_id': 773592320,
   'channel_id': 850025330,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950931534'
 annotations: {'unit_id': 950931534,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929433'
 annotations: {'unit_id': 950929433,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929417'
 annotations: {'unit_id': 950929417,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950931556'
 annotations: {'unit_id': 950931556,
   'probe_id':

In [68]:
list(block.segments[0].spiketrains[:5] + block.segments[0].spiketrains[6:])

[SpikeTrain
 name: '950929283'
 annotations: {'unit_id': 950929283,
   'probe_id': 773592320,
   'channel_id': 850025328,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929267'
 annotations: {'unit_id': 950929267,
   'probe_id': 773592320,
   'channel_id': 850025328,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929299'
 annotations: {'unit_id': 950929299,
   'probe_id': 773592320,
   'channel_id': 850025330,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950931534'
 annotations: {'unit_id': 950931534,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929433'
 annotations: {'unit_id': 950929433,
   'probe_id': 773592320,
   'channel_id': 850025334,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950931556'
 annotations: {'unit_id': 950931556,
   'probe_id': 773592320,
   'channel_id': 850025338,
   'brain_structure': 'VISp'},
 SpikeTrain
 name: '950929451'
 annotations: {'unit_id': 950929451,
   'probe_id':

In [57]:
len(block.segments[0].spiketrains[0])/block.segments[0].spiketrains[0].duration.magnitude

23.006378062924792

In [60]:
help(block.segments[0].spiketrains)

Help on SpikeTrainList in module neo.core.spiketrainlist object:

class SpikeTrainList(builtins.object)
 |  SpikeTrainList(items=None, segment=None)
 |  
 |  This class contains multiple spike trains, and can represent them
 |  either as a list of SpikeTrain objects or as a pair of arrays
 |  (all spike times in a single array, with a second array indicating which
 |  neuron/channel the spike is from).
 |  
 |  A SpikeTrainList object should behave like a list of SpikeTrains
 |  for iteration and item access. It is not intended to be used directly
 |  by users, but is available as the attribute `spiketrains` of Segments.
 |  
 |  Examples:
 |  
 |      # Create from list of SpikeTrain objects
 |  
 |      >>> stl = SpikeTrainList(items=(
 |      ...     SpikeTrain([0.5, 0.6, 23.6, 99.2], units="ms", t_start=0 * pq.ms, t_stop=100.0 * pq.ms),
 |      ...     SpikeTrain([0.0007, 0.0112], units="s", t_start=0 * pq.ms, t_stop=100.0 * pq.ms),
 |      ...     SpikeTrain([1100, 88500], units="

In [47]:
help(block.segments[0].spiketrains[0])

Help on SpikeTrain in module neo.core.spiketrain object:

class SpikeTrain(neo.core.dataobject.DataObject)
 |  SpikeTrain(times, t_stop, units=None, dtype=None, copy=True, sampling_rate=array(1.) * Hz, t_start=array(0.) * s, waveforms=None, left_sweep=None, name=None, file_origin=None, description=None, array_annotations=None, **annotations)
 |  
 |  :class:`SpikeTrain` is a :class:`Quantity` array of spike times.
 |  
 |  It is an ensemble of action potentials (spikes) emitted by the same unit
 |  in a period of time.
 |  
 |  *Usage*::
 |  
 |      >>> from neo.core import SpikeTrain
 |      >>> from quantities import s
 |      >>>
 |      >>> train = SpikeTrain([3, 4, 5]*s, t_stop=10.0)
 |      >>> train2 = train[1:3]
 |      >>>
 |      >>> train.t_start
 |      array(0.0) * s
 |      >>> train.t_stop
 |      array(10.0) * s
 |      >>> train
 |      <SpikeTrain(array([ 3.,  4.,  5.]) * s, [0.0 s, 10.0 s])>
 |      >>> train2
 |      <SpikeTrain(array([ 4.,  5.]) * s, [0.0 s, 10.0 

In [43]:
block.segments[0].spiketrains[0].annotations['unit_id']

950929283

In [33]:
dir_input = f"{PROJECT_PATH}/data/spike_stats/unit_rates"
df_spont = pd.read_csv(f"{dir_input}/spontaneous_stationary.csv").assign(stim='spont')

df_movie = pd.read_csv(f"{dir_input}/natural_movie_one_more_repeats.csv").assign(stim='movie')
df_shuffled = pd.read_csv(f"{dir_input}/natural_movie_one_shuffled.csv").assign(stim='shuffled')
df_stim = pd.concat([df_movie, df_shuffled])

In [34]:
df_spont.head()

,session,brain_structure,epoch_idx,unit_index,epoch_times,running,firing_rate,stim
0,767871931,VISp,0,0,"[4398.0, 4428.0]",False,0.333333,spont
1,767871931,VISp,0,1,"[4398.0, 4428.0]",False,3.333333,spont
2,767871931,VISp,0,2,"[4398.0, 4428.0]",False,0.700000,spont
3,767871931,VISp,0,3,"[4398.0, 4428.0]",False,1.533333,spont
4,767871931,VISp,0,4,"[4398.0, 4428.0]",False,8.600000,spont


In [35]:
df_stim.head()

,session,brain_structure,epoch_idx,unit_index,epoch_times,running,firing_rate,stim
0,766640955,VISp,0,0,"[2297.212884154053, 2327.2200000000003]",False,9.933333,movie
1,766640955,VISp,0,1,"[2297.212884154053, 2327.2200000000003]",False,4.866667,movie
2,766640955,VISp,0,2,"[2297.212884154053, 2327.2200000000003]",False,2.300000,movie
3,766640955,VISp,0,3,"[2297.212884154053, 2327.2200000000003]",False,0.533333,movie
4,766640955,VISp,0,4,"[2297.212884154053, 2327.2200000000003]",False,4.866667,movie


In [32]:
df_stim = df_stim[~df_stim['running']]

In [ ]:
def filter_units(df):
    # filter units of a df, comparing between states and then drop in place